In [1]:
# Config (controls parameters, based on whether running locally or on google colab)
try:
  import google.colab
  IN_COLAB = True
  path_prefix = ''
except:
  IN_COLAB = False
  path_prefix = 'data/'

TRAIN = True # Whether to train the model or not.
EXISTING_MODEL = '' # Path to an existing model checkpoint.
BATCH_SIZE = 1 # Set lower when using a GPU with less memory.

In [2]:
# Python libaries
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install tensorflow
    !pip install tokenizers
    !pip install nlpaug
    !pip install simpletransformers
    #!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.9 MB/s 
     |████████████████████████████████| 596 kB 47.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.0 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.8 MB/s 
     |████████████████████████████████| 212 kB 63.9 MB/s 
     |████████████████████████████████| 140 kB 66.0 MB/s 
     |████████████████████████████████| 86 kB 8.3 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |█████████████████

In [3]:
from datasets import load_dataset, concatenate_datasets, load_metric, Dataset 
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoConfig
import tqdm
import torch
import csv
import datetime
from IPython.display import display, HTML
import nltk
import nlpaug.augmenter.word as naw
from torch.utils.data import Dataset
import random
from sklearn import model_selection, feature_extraction
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import time
import warnings
from torch.utils.data import TensorDataset, DataLoader, Dataset as Ds
from typing import List, Tuple

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import KFold

In [4]:
# Source https://nlp.cs.princeton.edu/SARC/
!wget -nc https://nlp.cs.princeton.edu/SARC/0.0/key.csv
!wget -nc https://nlp.cs.princeton.edu/SARC/0.0/main/test-balanced.csv.bz2
!wget -nc https://nlp.cs.princeton.edu/SARC/0.0/main/train-balanced.csv.bz2
!bzip2 -dk test-balanced.csv.bz2
!bzip2 -dk train-balanced.csv.bz2

--2022-06-13 00:32:21--  https://nlp.cs.princeton.edu/SARC/0.0/key.csv
Resolving nlp.cs.princeton.edu (nlp.cs.princeton.edu)... 128.112.136.61
Connecting to nlp.cs.princeton.edu (nlp.cs.princeton.edu)|128.112.136.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80 [text/csv]
Saving to: ‘key.csv’

key.csv             100%[===================>]      80  --.-KB/s    in 0s      

2022-06-13 00:32:21 (9.42 MB/s) - ‘key.csv’ saved [80/80]

--2022-06-13 00:32:21--  https://nlp.cs.princeton.edu/SARC/0.0/main/test-balanced.csv.bz2
Resolving nlp.cs.princeton.edu (nlp.cs.princeton.edu)... 128.112.136.61
Connecting to nlp.cs.princeton.edu (nlp.cs.princeton.edu)|128.112.136.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20261578 (19M) [application/x-bzip2]
Saving to: ‘test-balanced.csv.bz2’

test-balanced.csv.b 100%[===================>]  19.32M  47.1MB/s    in 0.4s    

2022-06-13 00:32:22 (47.1 MB/s) - ‘test-balanced.csv.bz2’ saved [202615

In [5]:
# Load data
header_names = pd.read_csv('key.csv', sep='\t').columns.values.tolist()

dataset = load_dataset('csv', data_files='train-balanced.csv', sep='\t', names=header_names, split='train')

Using custom data configuration default-d313a5ac64883195


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d313a5ac64883195/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [ ]:
#None Comment
#dataset[56269]

In [6]:
from transformers import BertTokenizer, BertModel
def preprocess(row):
    new_text = []
    text = row['comment']
    for t in text or []:
      if t is not None:
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return {**row, 'comment':" ".join(new_text)}

def cleanup(d_set):
  return d_set.map(preprocess).remove_columns(['author','subreddit', 'score', 'ups', 'downs', 'date', 'created_utc'])

# Sourced from https://huggingface.co/cardiffnlp/twitter-roberta-base-mar2022
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["comment"], padding="max_length", truncation=True)


def tokenize(d_set):
    tokenized = d_set.map(tokenize_function, batched=True)
    combine = [f'{p_c}{tokenizer.sep_token}{c}' for p_c, c in tqdm.tqdm(zip(tokenized['parent_comment'], tokenized['comment']))]
    return tokenized.add_column('comment_pair', combine).remove_columns(['comment','parent_comment'])

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
# Split off validation data #no-GPU testing: test_size=0.99997 (30 samples) / 0.95 (50541 samples) / 0.990101 (10000 samples)
train_validation = dataset.train_test_split(test_size=0.995101, seed=777)
reduced_dataset = train_validation['train']
waste_dataset = train_validation['test']

reduced_dataset

Dataset({
    features: ['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc', 'parent_comment'],
    num_rows: 4952
})

In [8]:
# Basic classification accuracy metric
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
metrics = {
    'accuracy': accuracy_metric,
    'precision': precision_metric,
    'recall': recall_metric,
    'f1': f1_metric
}


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(predictions, labels)
    metrics = {
    'accuracy': accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
    'precision': precision_metric.compute(predictions=predictions, references=labels, zero_division=0)['precision'],
    'recall': recall_metric.compute(predictions=predictions, references=labels, zero_division=0)['recall'],
    'f1': f1_metric.compute(predictions=predictions, references=labels)['f1'],
    }
    return metrics

In [9]:
# Hyperparameter Settings
training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  logging_strategy="epoch",
                                  save_strategy="epoch",
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE)

In [10]:
start = time.time()
def k_fold_cross_validation(trainable_set ,k, training_args, compute_metrics):
    # First make the kfold object
    kf = KFold(n_splits=k, random_state=777, shuffle=True)

    data_set = pd.DataFrame(trainable_set)

    results = []

    for train_index, val_index in kf.split(data_set):
        # splitting Dataframe (dataset not included)
        train_df = trainable_set.select(train_index)
        val_df = trainable_set.select(val_index)

        # Defining Model
        config = AutoConfig.from_pretrained("bert-base-cased")
        model = AutoModelForSequenceClassification.from_config(config)

        tokenized_train = tokenize(cleanup(train_df))
        tokenize_validation = tokenize(cleanup(val_df))

        #"""
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train,
            eval_dataset=tokenize_validation,
            compute_metrics=compute_metrics,
            )
        if TRAIN:
          trainer.train()
        #"""
    print('Done')
    print(f'---------------------------------------------------------------------------------------------------------')

k_fold_cross_validation(trainable_set=reduced_dataset, k=5,training_args=training_args, compute_metrics=compute_metrics)

end = time.time()
print(f'took {round(end - start, 2)}s')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0/3961 [00:00<?, ?ex/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

3961it [00:00, 717682.76it/s]


  0%|          | 0/991 [00:00<?, ?ex/s]

...

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

Saving model checkpoint to test_trainer/checkpoint-7924
Configuration saved in test_trainer/checkpoint-7924/config.json
Model weights saved in test_trainer/checkpoint-7924/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: comment_pair. If comment_pair are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 990
  Batch size = 1


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

Saving model checkpoint to test_trainer/checkpoint-11886
Configuration saved in test_trainer/checkpoint-11886/config.json
Model weights saved in test_trainer/checkpoint-11886/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




Done
---------------------------------------------------------------------------------------------------------
took 8787.09s
